In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes

In [2]:
from nltk import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import extract_unigram_feats, mark_negation

C:\Users\Tejal\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
from unidecode import unidecode

In [4]:
from nltk.corpus import stopwords

In [5]:
s = set(stopwords.words('english'))
vec = TfidfVectorizer(use_idf=True, lowercase=True,strip_accents='ascii',stop_words=s )

In [16]:
data = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\Sentiment analysis\Train1\Train1.csv',names = ["Sentiment", "c1", "c2", "c3","c4","tweet"],encoding='latin-1')

In [244]:
data.groupby('Sentiment').count()

,c1,c2,c3,c4,tweet
Sentiment,,,,,
0,800000,800000,800000,800000,800000
4,800000,800000,800000,800000,800000


In [17]:
train_data = data[["Sentiment","tweet"]].copy()

In [13]:
train_data.shape

(1600000, 2)

In [8]:
import re

In [18]:
train_data['tweet']=train_data['tweet'].apply(lambda x: re.sub(r"http\S+", "", x))
train_data['tweet']=train_data['tweet'].apply(lambda x: re.sub(r"^@\S+", "", x))

In [24]:
train_data.head()

,Sentiment,tweet
0,0,"- Awww, that's a bummer. You shoulda got Da..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [20]:
import sklearn.utils
train_data = sklearn.utils.shuffle(train_data)
train_data = train_data.reset_index(drop=True)

In [21]:
train_data1 = train_data.iloc[0:40000].copy()

In [22]:
train_data1.groupby('Sentiment').count()

,tweet
Sentiment,
0,20090
4,19910


In [24]:
def clean_text(text):
    text = text.replace("<br />", " ")
    text = text.decode("utf-8")
 
    return text
clean_text(train_data.iloc[0,1])

u' haha likewise.  I always forget how different our time zones are. Perhaps I shall hunt you down on facebook now. '

In [33]:
analyzer = SentimentAnalyzer()
voc = analyzer.all_words([mark_negation(word_tokenize(unidecode(clean_text(instance.encode('ascii', 'ignore').decode('ascii').encode("utf-8")))))
                                 for instance in train_data1.iloc[:,1]])
print "Computing Unigran Features ..."
unigram_features = analyzer.unigram_word_feats(voc, min_freq=10)


Computing Unigran Features ...


In [35]:
analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_features)

In [36]:
train_X = analyzer.apply_features([mark_negation(word_tokenize(unidecode(clean_text(instance.encode('ascii', 'ignore').decode('ascii').encode("utf-8"))))) 
                                    for instance in train_data1.iloc[:,1]], labeled=False)

In [37]:
trainer = NaiveBayesClassifier.train
classifier = analyzer.train(trainer, zip(train_X, train_data1.iloc[:,0]))

Training classifier


In [124]:
acc_data = pd.read_csv(r'C:\Users\Tejal\Documents\Tejal\College\SSCI580\Project\all_usa_tweets.csv',encoding='latin-1')

In [125]:
acc_data['Tweet content']=acc_data['Tweet content'].apply(lambda x: re.sub(r"http\S+", "", x))
acc_data['Tweet content']=acc_data['Tweet content'].apply(lambda x: re.sub(r"^@\S+", "", x))

In [126]:
acc_data["neg"]=""
acc_data["neu"]=""
acc_data["pos"]=""

In [127]:
actual_data = []
x = 0
y = 34441
for i in range(0,5):
    actual_data.append(acc_data.iloc[x:y,:])
    x=y+1
    y=y+y

In [140]:
34441*4

137764

In [136]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [137]:
fin_data= []
file_name = "senti_chunck_"
for i in range(len(actual_data)):
    data = actual_data[i].copy()
    for j in range(len(data)):
        s=analyser.polarity_scores(data.iloc[j,2]) 
        data.iloc[j,5]=s['neg']
        data.iloc[j,6]=s['neu']
        data.iloc[j,7]=s['pos']
    f = file_name+str(i)+".csv"
    data.to_csv(f,sep=",",encoding='latin-1')
    fin_data.append(data)    
    

In [92]:
df1 = pd.DataFrame(fin_data[0])

In [90]:
fin_data[0].groupby('Senti').count()

,Date,Hour,Tweet content,Latitude,Longitude
Senti,,,,,
0,2763,2763,2763,2763,2763
4,31678,31678,31678,31678,31678


In [98]:
df1 = df1.append(fin_data[4])

In [100]:
df1.to_csv('full_us_senti.csv',sep=',',encoding='latin-1')